In [14]:
!pip install statsmodels

You should consider upgrading via the 'pip install --upgrade pip' command.


In [25]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tools.eval_measures import mse, rmse

#from sklearn.metrics import mean_squared_error, r2_score

In [27]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()
 
table_name = '"FACT_CUSTOMER_ORDERS"'
 
sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

ModuleNotFoundError: No module named 'fosforml'